In [1]:
import pandas as pd
import numpy as np
from train_utils import *
from sklearn.metrics import root_mean_squared_error as rmse
from xgboost import XGBRegressor, DMatrix, cv
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
import lightgbm as lgb
import torch
from catboost import CatBoostRegressor, Pool, cv as catcv
from sklearn.impute import SimpleImputer
# from sklearn.feature_selection import RFE

Read csv

In [2]:
seed = 42
random.seed(seed)
df_train = pd.read_csv("train.csv", low_memory=False)
X = df_train.drop('price', axis=1)
y = df_train['price']
X, y = shuffle(X, y) ## shuffle dataset, maybe it helps?

X_submission = pd.read_csv("test.csv", low_memory=False)
print(f"length of the total train data: {len(df_train)}")
print(f"length of the submission data: {len(X_submission)}")

length of the total train data: 33538
length of the submission data: 17337


Data processing pipeline

In [3]:
## process on the entire train dataset
X_processed = data_process_pipeline(X, X_submission, min_freq=30)

total number of features generated: 15438
Length of processed data: 33538


In [5]:
X_submission_processed = data_process_pipeline(X_submission, X_submission, min_freq=35)

total number of features generated: 14111
Length of processed data: 17337


In [ ]:
# # X_processed.drop(columns={"host_acceptance_rate"}, axis=1, inplace=True)
# X_submission_processed.drop(columns={"host_acceptance_rate"}, axis=1, inplace=True)

xgboost

In [6]:
xgb_params = {
    # 'n_estimators': 1000,
    # 'num_parallel_tree': 1,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    # 'tree_method':"hist",
    # 'device': 'cuda',
    'max_depth': 7,
    # 'gamma': 0.5,
    # 'lambda': 100,
    # 'alpha': 10,
    'colsample_bytree': 0.8,
    'colsample_bynode': 0.7,
    'colsample_bylevel': 0.5,
    'min_child_weight': 10,
    'subsample':0.95,
    'learning_rate': 0.05,
}

In [7]:
## model evaluation: cross validation 
data = X_processed
label = y

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=seed)
dtrain = DMatrix(data=X_train, label=y_train, enable_categorical=True)
dtest = DMatrix(data=X_test, label=y_test, enable_categorical=True)
bst = xgb.train(xgb_params, dtrain, num_boost_round=850)
xgb_pred = bst.predict(dtest)
rmse(y_test, xgb_pred)
# 82.2739999451707 (prev best score)

# cv_results = cv(dtrain=dtrain, params=params, nfold=5, num_boost_round=1000, metrics='rmse', seed=seed, as_pandas=True)
# cv_results = cv(dtrain=dtrain, params=xgb_params, nfold=2, num_boost_round=1000, early_stopping_rounds=20, verbose_eval=True)
# cv_results = cv(dtrain=dtrain, params=params, nfold=5, num_boost_round=1000, early_stopping_rounds=20, metrics='rmse', seed=seed, as_pandas=True)
# cv_results[-10:] ## best: 80.55

82.23080901032674

In [9]:
## submission
dall = DMatrix(data=X_processed, label=y, enable_categorical=True)
dsub = DMatrix(data=X_submission_processed, enable_categorical=True)
sub_bst = xgb.train(xgb_params, dall, num_boost_round=850)
xgb_sub_preds = sub_bst.predict(dsub)
xgb_sub_preds

ValueError: feature_names mismatch: ['host_acceptance_rate', 'host_listings_count', 'bathrooms', 'bedrooms', 'beds', 'square_feet', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'extra_people', 'host_response_rate', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable', 'is_business_travel_ready', 'require_guest_profile_picture', 'require_guest_phone_verification', 'cancellation_policy', 'host_response_time', 'first_review_year', 'first_review_month', 'first_review_day', 'last_review_year', 'last_review_month', 'last_review_day', 'host_since_year', 'host_since_month', 'host_since_day', 'state', 'country', 'property_type', 'room_type', 'city', 'neighbourhood_cleansed', 'host_location', 'host_neighbourhood', 'neighbourhood_group_cleansed', 'market', 'bed_type', 'amenities_nans/empty', 'amenities_

LightGBM (Light Gradient Boosting Machine)

In [8]:
lgb_params = params = {
    'objective': 'regression',  # 'reg:squarederror' in XGBoost is simply 'regression' in LightGBM.
    'metric': 'rmse',  # 'eval_metric' in XGBoost is 'metric' in LightGBM for evaluation metrics.
    # 'tree_method' and 'device' are specific to XGBoost for specifying the algorithm and hardware. LightGBM automatically uses the best method based on the data size and can be controlled by 'device_type' for GPU usage.
    # For GPU usage in LightGBM, you would set 'device_type': 'gpu'.
    # 'device_type': 'gpu', 
    'max_depth': 7,
    # 'min_split_gain': 1, # in XGBoost is similar to 'min_split_gain' in LightGBM.
    # 'lambda' in XGBoost is 'lambda_l2' in LightGBM for L2 regularization.
    # 'alpha' in XGBoost is 'lambda_l1' in LightGBM for L1 regularization.
    'colsample_bytree': 0.8,  # This is the same in LightG|BM.
    'feature_fraction_bynode': 0.7,  # LightGBM does not have an exact equivalent;
    # 'feature_fraction_by_level': 0.5,  # In LightGBM, it's 'feature_fraction_by_level'.  doesn't work
    'min_child_samples': 20,  # In LightGBM, this is 'min_child_samples' for the minimum number of data in a leaf.
    'learning_rate': 0.05,  # This is the same in LightGBM.
    # Additional adjustments for LightGBM
    'num_leaves': 36 ## You might want to set this as it's crucial in LightGBM for controlling tree complexity.
    # 'bagging_freq': 2, # Needed if you're using 'subsample'. Set to 1 to enable bagging.
    # 'bagging_fraction': 0.85,  # In LightGBM, it's 'bagging_fraction', aka subsample.
}

In [ ]:
%%time
data = X_processed
# X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=seed)
lgb_dtrain = lgb.Dataset(X_train, label=y_train)

lgb_model = lgb.train(train_set=lgb_dtrain, params=lgb_params, num_boost_round=850)
lgb_pred = lgb_model.predict(X_test)
rmse(y_test, lgb_pred)
# 81.94895211779131

# lgb_dall = lgb.Dataset(data, label=y)
# cv_results = lgb.cv(train_set=lgb_dall, nfold=5, params={**lgb_params, "early_stopping_round":15}, num_boost_round=850, seed=seed)
# cv_results['valid rmse-mean'][-5:]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.661189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31284
[LightGBM] [Info] Number of data points in the train set: 23476, number of used features: 14939
[LightGBM] [Info] Start training from score 144.731556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

81.94895211779131

In [ ]:
lgb_dall = lgb.Dataset(X_processed, label=y_train)
lgb_sub_model = lgb.train(train_set=lgb_dall, params=lgb_params, num_boost_round=850)
lgb_sub_pred = lgb_sub_model.predict(X_submission_processed)
lgb_sub_pred

81.94895211779131

Catboost Meow

In [ ]:
cname = [x for x in X_processed.columns[X_processed.dtypes == "category"]]
cat_params = {
    "loss_function": "RMSE",
    'iterations': 850,
    'learning_rate': 0.1,
    'depth': 7,
    'l2_leaf_reg':0.2,
    'verbose': True,
    # "task_type": "GPU",
    "random_strength": 5,
    # 'l2_leaf_reg': None, # Equivalent to 'lambda_l2' in LightGBM for L2 regularization. Not specified in your LightGBM params, so it's commented out.
    # 'l1_leaf_reg': None, # Equivalent to 'lambda_l1' in LightGBM for L1 regularization. Not specified in your LightGBM params, so it's commented out.
    'colsample_bylevel': 0.8,  # Approximate equivalent to 'colsample_bytree' in LightGBM.
    # There is no direct equivalent to 'feature_fraction_bynode' in CatBoost.
    'min_data_in_leaf': 20,  # Approximate equivalent to 'min_child_samples' in LightGBM for the minimum number of data in a leaf.
    'learning_rate': 0.05,  # This is the same in CatBoost.
    # Additional CatBoost-specific parameters
    # 'num_leaves': None, # CatBoost does not use 'num_leaves'. It controls tree complexity with 'depth'.
    # 'bootstrap_type': 'Bernoulli', # If you're using bagging, CatBoost uses 'bootstrap_type'. 'Bayesian', 'Bernoulli', 'MVS', and 'No' are options.
    'subsample': 0.90, # If 'bootstrap_type' is set to 'Bernoulli', 'subsample' is the equivalent to 'bagging_fraction' in LightGBM.
    'bootstrap_type': 'Bernoulli'
}

pool_train = Pool(X_train, y_train, cat_features=cname)
pool_test = Pool(X_test, y_test, cat_features=cname)
cb_model = CatBoostRegressor(**cat_params)
cb_model.fit(pool_train)
cb_preds = cb_model.predict(pool_test)
rmse(y_test, cb_preds)
# 83.3347878304928

# cat_cv = catcv(pool, cat_params, fold_count=5)
# cat_cv["test-RMSE-mean"].iloc[-1]

In [ ]:
cb_sub_model = CatBoostRegressor(**cat_params)
pool = Pool(X_processed, y, cat_features=cname)
cb_sub_model.fit(pool)
final_pool = Pool(X_submission_processed, cat_features=cname)
cb_sub_pred = cb_sub_model.predict(final_pool)
cb_sub_pred

array([580.86057964, 148.16715608,  87.63142184, ...,  79.29511443,
        53.51821195, 124.08984595])

Random Forest

In [95]:
rf_params = {
    'n_estimators': 1000,
    # 'max_depth': , 
    'min_samples_split': 20, 
    'min_samples_leaf': 20, 
    'min_weight_fraction_leaf': 0.05, 
    # 'min_impurity_decrease': 0.5,
    # 'max_features': , 
    # 'bootstrap': , 
    # 'max_samples': , 
    'verbose':1
} 

In [93]:
imputer = SimpleImputer(strategy="median")


X_train_float = X_train.select_dtypes(include=['float64', 'float32'])
X_train_nonfloat = X_train.select_dtypes(exclude=['float64', 'float32'])
X_train_imputed_float = imputer.fit_transform(X_train_float)
X_imputed_train_dict = {X_train_float.columns[i]: X_train_imputed_float[:, i] for i in range(X_train_float.shape[1])}
X_imputed_train = pd.DataFrame(X_imputed_train_dict)

X_test_float = X_test.select_dtypes(include=['float64', 'float32'])
X_test_nonfloat = X_test.select_dtypes(exclude=['float64', 'float32'])
X_test_imputed_float = imputer.fit_transform(X_test_float)
X_imputed_test_dict = {X_test_float.columns[i]: X_test_imputed_float[:, i] for i in range(X_test_float.shape[1])}
X_imputed_test = pd.DataFrame(X_imputed_test_dict)

In [96]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(**rf_params, random_state=42)

rf.fit(X_train_imputed_float, y_train)
y_pred = rf.predict(X_test_imputed_float)
rmse(y_test, y_pred)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   32.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.4s finished


113.19691411089967

In [17]:
ensemble_pred = 0.3 * xgb_pred + 0.4 * lgb_pred + 0.3 * cb_preds
rmse(y_test, ensemble_pred)
# 81.24 (prev best)

# ensemble_sub_pred = 

81.24074587525985

In [27]:
# pred_df = {"Id": X_submission.id, "Predicted": cb_pred}
# cb_df = pd.DataFrame(pred_df)
# cb_df.to_csv("cb_submission.csv", index=False)

SVM

Ensemble Tuning & Saving Submission to csv

In [28]:
# lgbm_pred = pd.read_csv("lgbm_submission.csv")
# lgbm_pred["Predicted"]

0        538.657907
1        154.971403
2         41.181560
3         49.908280
4         45.937114
            ...    
17332     62.410054
17333    290.939147
17334    119.211017
17335     51.258311
17336    129.996408
Name: Predicted, Length: 17337, dtype: float64

In [38]:
# saving ensemble
## 1st place official formula: ensemble_red = 0.7 * lgbm_pred["Predicted"] + 0.3 * cb_pred
pred_df = {"Id": X_submission.id, "Predicted": ensemble_red}
ensemble_df = pd.DataFrame(pred_df)
ensemble_df.to_csv("ensemble_submission.csv", index=False)

TODO:
<!-- 1. Mimic previous score with train_utils function (make sure it works) -->
<!-- 2. Create a test split and see if there is indeed a big discrepency between cross validation and test score -->

<!-- 3.5 Try cleaning up words using spacy or ntlk -->

3.6 Try catboost and then ensemble

4. Use ALBERT to create new features